In [92]:
import numpy as np
import sys
import math
import time
import random
from typing import Callable

ListOfListType = list[list[int or float]]
ObjetiveType = [[ListOfListType, ListOfListType], int or float]

class AntColony(object):

    def __init__(self, number_ants = 5, matrixA = [], matrixB = [], iterations = 20, ro = 0.5, alpha = 1, betha = 1):
        self.number_ants = number_ants
        self.matrixA = matrixA
        self.matrixB = matrixB
        self.iterations = iterations
        self.ro = ro
        self.alpha = alpha
        self.betha = betha

    def create_nodes(self) -> list:
        first_solution = list(range(1, (self.matrixA.shape[0] + 1)))
        random.shuffle(first_solution)
        return first_solution

    def create_dictionary(self, size: int, value: int) -> dict:
        dictionary = dict()
        for i in np.arange(size):
            for j in np.arange(size):
                if(i < j):
                    key = str(i+1) + str(j+1)
                    dictionary.setdefault(key, value)
        return dictionary

    def init_pheromons(self, solution: list, value: int) -> dict:
        pheromons = self.create_dictionary(size=len(solution), value=value)
        return pheromons

    def calculate_costs(self, nodes: list, actual_solution: list, index:int) ->list:
        costs = []

        for node in nodes:
            new_solution = actual_solution.copy()
            new_solution[index] = node
            costs.append(self.cost_function(self.matrixA, self.matrixB, new_solution) + 1)
        return costs

    def calculate_probabilitys(self,nodes: list, index: int, actual_solution: list, pheromons: dict, costs: list):
        actual_node = actual_solution[index]
        probabilitys = []
        divisor = 0

        for i,node in enumerate(nodes):
            if(node):
                if(actual_node < node):
                    a1, a2 = actual_node, node
                else:
                    a1, a2 = node, actual_node
                
                key = str(a1) + str(a2)
                pherom = pheromons[key]
                divisor += (math.pow(pherom, self.alpha)) * (math.pow((1 / costs[i]), self.betha))

        for i,node in enumerate(nodes):
            if(node):
                if(actual_node < node):
                    a1, a2 = actual_node, node
                else:
                    a1, a2 = node, actual_node
                
                key = str(a1) + str(a2)
                pherom = pheromons[key]
    
            probabilitys.append((math.pow(pherom, self.alpha) * math.pow(((1 / costs[i]) / divisor), self.betha)))
        
        return probabilitys
    
    def roulette_method(self, costs: ListOfListType) -> ListOfListType:
        sort_costs = np.array(costs)
        sort_costs = np.sort(costs)
        cost_index = -1

        winner = random.uniform(0,1)
        
        porcentage = 0 
        for i,proportion in enumerate(sort_costs):
            porcentage += proportion
            if(winner <= porcentage):
                cost_index = i
                break
        index_winner = costs.index(sort_costs[cost_index])
        return index_winner

    def based_on_position(self, costs: ListOfListType) -> ListOfListType:
        sort_costs = np.array(costs)
        sort_costs = np.sort(costs)

        proportions = []

        #print(sort_costs)

        mean_rank = (len(sort_costs) * (len(sort_costs) + 1)) / 2 

        for i,cost in enumerate(sort_costs):
            proportions.append((len(sort_costs) - i) / mean_rank )
        
        winner = random.uniform(0,1)
        
        sum_proportions = 0
        for i, proportion in enumerate(proportions):
            sum_proportions += proportion
            if(winner < sum_proportions):
                index_winner = i
                break
        index_winner = costs.index(sort_costs[index_winner])

        return index_winner


    def build_solution(self, solution: list, pheromons: dict):
        actual_solution = lista_ceros = np.zeros(self.matrixA.shape[0]).astype('int').tolist()
        index = 0
        nodes = solution.copy()
        sum_pheromons = 0
        
        initial_node = random.choice(nodes)
        actual_solution[index] = initial_node
        nodes.remove(initial_node)

        while(len(nodes) > 0):
            costs = self.calculate_costs(nodes=nodes, actual_solution=actual_solution, index=index)
            probabilitys = self.calculate_probabilitys(nodes=nodes, index=index, actual_solution=actual_solution, pheromons=pheromons, costs=costs)
            index_winner = self.roulette_method(costs=probabilitys)
            sum_pheromons += costs[index_winner]

            index += 1
            actual_solution[index] = nodes[index_winner]
            nodes.remove(nodes[index_winner])
        return [self.cost_function(self.matrixA, self.matrixB, actual_solution), actual_solution, sum_pheromons]

    def get_best_solution(self, solutions: ListOfListType) -> ListOfListType:
        sort_solutions = sorted(solutions, key=lambda x: x[0], reverse=False)

        return sort_solutions[0][0], sort_solutions[0][1]
    
    def evaporate_pheromons(self,pheromons: dict, ro:float, size: int):
        for i in np.arange(size):
            for j in np.arange(size):
                if(i < j):
                    key = str(i+1) + str(j+1)
                    pheromons[key] = pheromons[key] * (1 - ro)
                    

    def update_pheromons(self, results: list, pheromons:dict, actual_solution: list, ro: float):
        #new_pheromons = self.init_pheromons(solution=actual_solution, value=0)
        self.evaporate_pheromons(pheromons=pheromons, ro=ro, size=len(actual_solution))

        for result in results:
            solution = result[1]
            sum_pheromons = (1 / result[2])
            for i in np.arange(len(solution) - 1):
                if(solution[i] < solution[i+1]):
                    a1, a2 = solution[i], solution[i+1]
                else:
                    a1, a2 = solution[i+1], solution[i]
                key = str(a1) + str(a2)
                pheromons[key] = pheromons[key] + sum_pheromons
        return pheromons
                

    def fit(self, objetive: ObjetiveType):
        self.cost_function = objetive
        self.best_cost = float("inf")
        self.costs = []
        inicio = time.time()
        best_solution = []
        best_cost = float("inf")
        nodes = self.create_nodes()
        pheromons = self.init_pheromons(solution=nodes, value=1)

        i = 1
        while(i < self.iterations):
            solutions = []
            for _ in np.arange(self.number_ants):
                solutions.append(self.build_solution(solution=nodes, pheromons=pheromons))
            
            best_cost, best_solution = self.get_best_solution(solutions=solutions)

            if(best_cost < self.best_cost):
                self.best_cost = best_cost
                self.best_solution = best_solution
            self.costs.append(self.best_cost)
            
            pheromons = self.update_pheromons(results=solutions, actual_solution=actual_solution, pheromons=pheromons, ro=self.ro)
            
            fin = time.time()
            total_time = fin - inicio
            sys.stderr.write('\r%d Iteration: | Cost %f | Time: %f' % (i, self.best_cost, total_time))
            time.sleep(0)  
            sys.stderr.flush()
            i += 1
        self.best_cost = best_cost
        self.best_solution = best_solution

In [93]:
import csv

matrixA = []
with open('./data/n_12/chr12a.csv', 'r', newline='') as file:
  myreader = csv.reader(file, delimiter=',')
  for rows in myreader:
   matrixA.append(rows)
matrixA = np.array(matrixA).astype("float")

matrixB = []
with open('./data/n_12/chr12b.csv', 'r', newline='') as file:
  myreader = csv.reader(file, delimiter=',')
  for rows in myreader:
   matrixB.append(rows)
matrixB = np.array(matrixB).astype("float")

In [94]:
def costo(matrixA, matrixB, actual_solution):
    cost = 0
    for i in np.arange((len(matrixA))):
        for j in np.arange((len(matrixA))):      
            cost += matrixA[i][j] * matrixB[actual_solution[i]-1][actual_solution[j]-1]
    return cost

print(costo(matrixA, matrixB, [5,7,1,10,11,3,4,2,9,6,12,8]))

9742.0


In [95]:
ac = AntColony(matrixA=matrixA, matrixB=matrixB, number_ants=10, ro=2, alpha=1, betha=2)

ac.fit(costo)

{'12': 1, '13': 1, '14': 1, '15': 1, '16': 1, '17': 1, '18': 1, '19': 1, '110': 1, '111': 1, '112': 1, '23': 1, '24': 1, '25': 1, '26': 1, '27': 1, '28': 1, '29': 1, '210': 1, '211': 1, '212': 1, '34': 1, '35': 1, '36': 1, '37': 1, '38': 1, '39': 1, '310': 1, '311': 1, '312': 1, '45': 1, '46': 1, '47': 1, '48': 1, '49': 1, '410': 1, '411': 1, '412': 1, '56': 1, '57': 1, '58': 1, '59': 1, '510': 1, '511': 1, '512': 1, '67': 1, '68': 1, '69': 1, '610': 1, '611': 1, '612': 1, '78': 1, '79': 1, '710': 1, '711': 1, '712': 1, '89': 1, '810': 1, '811': 1, '812': 1, '910': 1, '911': 1, '912': 1, '1011': 1, '1012': 1, '1112': 1}


KeyboardInterrupt: 